In [136]:
import os
import pandas as pd
import numpy as np
from scipy.signal import butter, filtfilt
import sys
import contextlib

directory = 'filling' 


In [138]:
import os
import pandas as pd
from scipy.signal import butter, filtfilt

# Set the directory where the CSV files are stored
directory = 'filling'

# Define the Butterworth filter parameters
sampling_rate = 1  # Sample rate in Hz (1 sample per hour)
cutoff_freq = 1 / 48  # Cutoff frequency in Hz (48-hour low pass filter)
nyquist_freq = sampling_rate / 2  # Nyquist frequency
normalized_cutoff = cutoff_freq / nyquist_freq  # Normalized cutoff frequency

# Calculate the filter coefficients
b, a = butter(N=2, Wn=normalized_cutoff, btype='low', analog=False)

# Loop through each file path
for path in os.listdir(directory):
    if path.endswith('.csv'):
        file_path = os.path.join(directory, path)
        df = pd.read_csv(file_path, index_col=0, parse_dates=['Time'])
        
        # Linearly interpolate the 'Sl' column with forward fill (no zigzag)
        df['Sl'] = df['Sl'].interpolate(method='linear', limit_direction='forward')
        # Drop rows before the first non-NaN value
        first_valid_index = df['Sl'].first_valid_index()
        df = df.loc[first_valid_index:]

        # Apply the Butterworth low-pass filter to the 'Sl' column
        df['Sl'] = filtfilt(b, a, df['Sl'])
        
        # Save the interpolated and filtered DataFrame back to CSV if needed
        filtered_file_path = os.path.join(directory, 'filtered_' + path)
        df.to_csv(filtered_file_path,index=False)

print("Interpolation and filtering completed.")

Interpolation and filtering completed.
